# Pangeo - Global Surface Air Temperature changes in future scenarios

> This notebook is a reproducibility example of the IPCC-WGI AR6 Interactive Atlas products, which has been adapted to work with the ESGF Virtual Aggregation. This work is licensed under a [Creative Commons Attribution 4.0 International License](http://creativecommons.org/licenses/by/4.0).
>
> ![Creative Commons License >](https://i.creativecommons.org/l/by/4.0/88x31.png)

**E. Cimadevilla** (Santander Meteorology Group. Instituto de Física de Cantabria, CSIC-UC, Santander, Spain).

This notebook computes Global Surface Air Temperature (GSAT) changes relative to the 1995–2014 and 1850–1900 averages, according to CMIP6 climate models.

This notebook works with data available in the [Google Cloud CMIP6 dataset](https://console.cloud.google.com/marketplace/product/noaa-public/cmip6). The Google Cloud CMIP6 data are derived from the original CMIP6 data files, as distributed via the Earth System Grid Federation (ESGF). Consistent with the CMIP6 terms of use, some modifications have been made to render the data more analysis-ready, including concatenation of time slices and conversion from netCDF to [Zarr](https://zarr.readthedocs.io/) format. All relevant metadata, including information about how to cite, are provided in the zarr metadata. The CMIP6 hosted on Google Cloud are maintained by the Climate Data Science Lab at Lamont Doherty Earth Observatory (LDEO) of Columbia University, as part of the [Pangeo Project](https://pangeo.io/) .

In [ ]:
import requests

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cftime

import xarray
import dask

In [ ]:
dask.config.set(scheduler="processes")

## Data catalog

The `gsat-pangeo.csv` catalog include the paths to the Google Cloud datasets considered in this notebook. We can analyze it with Pandas.

In [ ]:
df = pd.read_csv("gsat-pangeo.csv")
df

## Data loading

We load the data into [xarray](https://docs.xarray.dev/en/stable/) datasets, since xarray is compatible with [Zarr](https://docs.xarray.dev/en/stable/user-guide/io.html#zarr).

In [ ]:
def load_datasets_pangeo(locations, experiment):
    dss = []
    for location in locations:
        if location is np.nan:
            continue

        model = location.split("/")[6]
        member = location.split("/")[8]

        ds = xarray.open_zarr(
            location,
            drop_variables=["time_bnds", "lat_bnds", "lon_bnds"])

        if experiment.startswith("ssp"):
            ds = ds.isel(time=slice(None, 31390))
            # limit to 2100, avoid cf time indexing issues
            if isinstance(ds["time"][0].item(), cftime.Datetime360Day):
                ds = ds.sel(time=slice("20150101", "21001230"))
            else:
                ds = ds.sel(time=slice("20150101", "21001231"))
        elif experiment == "historical":
            pass
        else:
            raise ValueError("Invalid experiment... exiting")
    
        # add the model_run global attribute
        ds.attrs["model_run"] = f"{model}_{member}"
        dss.append(ds)

    return dss

Load the different scenarios.

In [ ]:
historical_dss = load_datasets_pangeo(df["historical"], "historical")
ssp126_dss = load_datasets_pangeo(df["ssp126"], "ssp126")
ssp245_dss = load_datasets_pangeo(df["ssp245"], "ssp245")
ssp370_dss = load_datasets_pangeo(df["ssp370"], "ssp370")
ssp585_dss = load_datasets_pangeo(df["ssp585"], "ssp585")

Preview one of the datasets.

In [ ]:
historical_dss[0]["tas"]

## Computation

The functions that compute global surface average temperatures (GSAT) is available in the `gsat.py` module. We implement weighted and unweighted GSAT.

In [ ]:
from gsat import weighted, unweighted

### Weighted

Due to the implementation of weighted operations between xarray and Dask, this operation is more expensive in comparison with the unweighted version. This is due to Dask tasks that have to wait for each other in order to carry out weighted operations.


- 3h 9min 43s

In [ ]:
%%time

mean_hist_1995_2014, mean_hist_1850_1900, mean_hist, mean_ssp126, mean_ssp245, mean_ssp370, mean_ssp585 = weighted(
    8,
    historical_dss, ssp126_dss, ssp245_dss, ssp370_dss, ssp585_dss)

In [ ]:
m_hist   = (mean_hist   - mean_hist_1995_2014).resample({"time": "YE"}).mean().mean("member").isel(time=slice(None,-1))["tas"]
m_ssp126 = (mean_ssp126 - mean_hist_1995_2014).resample({"time": "YE"}).mean().mean("member").isel(time=slice(None,-1))["tas"]
m_ssp245 = (mean_ssp245 - mean_hist_1995_2014).resample({"time": "YE"}).mean().mean("member").isel(time=slice(None,-1))["tas"]
m_ssp370 = (mean_ssp370 - mean_hist_1995_2014).resample({"time": "YE"}).mean().mean("member").isel(time=slice(None,-1))["tas"]
m_ssp585 = (mean_ssp585 - mean_hist_1995_2014).resample({"time": "YE"}).mean().mean("member").isel(time=slice(None,-1))["tas"]

m_hist_18   = (mean_hist   - mean_hist_1850_1900).resample({"time": "YE"}).mean().mean("member").isel(time=slice(None,-1))["tas"]
m_ssp126_18 = (mean_ssp126 - mean_hist_1850_1900).resample({"time": "YE"}).mean().mean("member").isel(time=slice(None,-1))["tas"]
m_ssp245_18 = (mean_ssp245 - mean_hist_1850_1900).resample({"time": "YE"}).mean().mean("member").isel(time=slice(None,-1))["tas"]
m_ssp370_18 = (mean_ssp370 - mean_hist_1850_1900).resample({"time": "YE"}).mean().mean("member").isel(time=slice(None,-1))["tas"]
m_ssp585_18 = (mean_ssp585 - mean_hist_1850_1900).resample({"time": "YE"}).mean().mean("member").isel(time=slice(None,-1))["tas"]

q_hist   = (mean_hist   - mean_hist_1995_2014).resample({"time": "YE"}).mean().quantile([.05, .95], dim=["member"]).isel(time=slice(None,-1))["tas"]
q_ssp126 = (mean_ssp126 - mean_hist_1995_2014).resample({"time": "YE"}).mean().quantile([.05, .95], dim=["member"]).isel(time=slice(None,-1))["tas"]
q_ssp370 = (mean_ssp370 - mean_hist_1995_2014).resample({"time": "YE"}).mean().quantile([.05, .95], dim=["member"]).isel(time=slice(None,-1))["tas"]
q_ssp585 = (mean_ssp585 - mean_hist_1995_2014).resample({"time": "YE"}).mean().quantile([.05, .95], dim=["member"]).isel(time=slice(None,-1))["tas"]

In [ ]:
fig, ax = plt.subplots(figsize=(5,5))

ax.fill_between(q_hist.time,
                q_hist.sel(quantile=.05),
                q_hist.sel(quantile=.95),
                color="#616161", alpha=0.25)
ax.fill_between(q_ssp126.time,
                q_ssp126.sel(quantile=.05),
                q_ssp126.sel(quantile=.95),
                color="#0e57cc", alpha=0.25)
ax.fill_between(q_ssp370.time,
                q_ssp370.sel(quantile=.05),
                q_ssp370.sel(quantile=.95),
                color="#e81515", alpha=0.25)

ytop = 6
ax.set_ylabel("Relative to 1995-2014 (°C)")
ax.axhline(y=0, color="black")
ax.text(np.datetime64('2024-12-31T00:00:00.000000000'), -1, "Near\nterm")
ax.text(np.datetime64('2044-12-31T00:00:00.000000000'), -1, "Mid\nterm")
ax.text(np.datetime64('2084-12-31T00:00:00.000000000'), -1, "Long\nterm")
ax.fill_between(
    m_ssp126["time"].sel(time=slice("20200101", "20400101")),
    -2, ytop+.82, alpha=.33, color="grey")
ax.fill_between(
     m_ssp126["time"].sel(time=slice("20400101", "20600101")),
    -2, ytop+.82, alpha=.33, color="grey")
ax.fill_between(
     m_ssp126["time"].sel(time=slice("20800101", None)),
    -2, ytop+.82, alpha=.33, color="grey")

ax2 = ax.twinx()
ax2.set_ylabel("Relative to 1850-1900 (°C)")
ax2.set_ylim(bottom=-2+.82, top=ytop+.82)
ax.set_ylim([-2, ytop])

ax.plot(m_hist.time, m_hist, color="#010101", label=f"Historical  {len(mean_hist['member'])}")
ax.plot(m_ssp126.time, m_ssp126, color="#0e57cc", label=f"SSP1-2.6  {len(mean_ssp126['member'])}")
ax.plot(m_ssp245.time, m_ssp245, color="#e68d26", label=f"SSP2-4.5  {len(mean_ssp245['member'])}")
ax.plot(m_ssp370.time, m_ssp370, color="#e81515", label=f"SSP3-7.0  {len(mean_ssp370['member'])}")
ax.plot(m_ssp585.time, m_ssp585, color="#9b1a22", label=f"SSP5-8.5  {len(mean_ssp585['member'])}")

# ax2.plot(m_hist_18.time, m_hist_18, color="#010101", label=f"Historical  {len(mean_hist['member'])}")
# ax2.plot(m_ssp126_18.time, m_ssp126_18, color="#0e57cc", label=f"SSP1-2.6  {len(mean_ssp126['member'])}")
# ax2.plot(m_ssp245_18.time, m_ssp245_18, color="#e68d26", label=f"SSP2-4.5  {len(mean_ssp245['member'])}")
# ax2.plot(m_ssp370_18.time, m_ssp370_18, color="#e81515", label=f"SSP3-7.0  {len(mean_ssp370['member'])}")
# ax2.plot(m_ssp585_18.time, m_ssp585_18, color="#9b1a22", label=f"SSP5-8.5  {len(mean_ssp585['member'])}")

ax.margins(0)
ax2.margins(0)
ax.legend()

plt.savefig("pangeo-weighted.pdf")

### Unweighted

In contrast to the weighted version, unweighted tasks in the Dask graph do not depend on each other for the computation of the final result. Thus, we can measure the actual throughtput of the infrastructure, since we are not spending time within the computation runtime.

260 GB

- 1h 30min 58s
- 1h 31min 7s

In [ ]:
%%time

un_mean_hist_1995_2014, un_mean_hist_1850_1900, un_mean_hist, un_mean_ssp126, un_mean_ssp245, un_mean_ssp370, un_mean_ssp585 = unweighted(
    8,
    historical_dss, ssp126_dss, ssp245_dss, ssp370_dss, ssp585_dss)

In [ ]:
m_hist   = (un_mean_hist   - un_mean_hist_1995_2014).resample({"time": "YE"}).mean().mean("member").isel(time=slice(None,-1))["tas"]
m_ssp126 = (un_mean_ssp126 - un_mean_hist_1995_2014).resample({"time": "YE"}).mean().mean("member").isel(time=slice(None,-1))["tas"]
m_ssp245 = (un_mean_ssp245 - un_mean_hist_1995_2014).resample({"time": "YE"}).mean().mean("member").isel(time=slice(None,-1))["tas"]
m_ssp370 = (un_mean_ssp370 - un_mean_hist_1995_2014).resample({"time": "YE"}).mean().mean("member").isel(time=slice(None,-1))["tas"]
m_ssp585 = (un_mean_ssp585 - un_mean_hist_1995_2014).resample({"time": "YE"}).mean().mean("member").isel(time=slice(None,-1))["tas"]

m_hist_18   = (un_mean_hist   - un_mean_hist_1850_1900).resample({"time": "YE"}).mean().mean("member").isel(time=slice(None,-1))["tas"]
m_ssp126_18 = (un_mean_ssp126 - un_mean_hist_1850_1900).resample({"time": "YE"}).mean().mean("member").isel(time=slice(None,-1))["tas"]
m_ssp245_18 = (un_mean_ssp245 - un_mean_hist_1850_1900).resample({"time": "YE"}).mean().mean("member").isel(time=slice(None,-1))["tas"]
m_ssp370_18 = (un_mean_ssp370 - un_mean_hist_1850_1900).resample({"time": "YE"}).mean().mean("member").isel(time=slice(None,-1))["tas"]
m_ssp585_18 = (un_mean_ssp585 - un_mean_hist_1850_1900).resample({"time": "YE"}).mean().mean("member").isel(time=slice(None,-1))["tas"]

q_hist   = (un_mean_hist   - un_mean_hist_1995_2014).resample({"time": "YE"}).mean().quantile([.05, .95], dim=["member"]).isel(time=slice(None,-1))["tas"]
q_ssp126 = (un_mean_ssp126 - un_mean_hist_1995_2014).resample({"time": "YE"}).mean().quantile([.05, .95], dim=["member"]).isel(time=slice(None,-1))["tas"]
q_ssp370 = (un_mean_ssp370 - un_mean_hist_1995_2014).resample({"time": "YE"}).mean().quantile([.05, .95], dim=["member"]).isel(time=slice(None,-1))["tas"]
q_ssp585 = (un_mean_ssp585 - un_mean_hist_1995_2014).resample({"time": "YE"}).mean().quantile([.05, .95], dim=["member"]).isel(time=slice(None,-1))["tas"]

In [ ]:
fig, ax = plt.subplots(figsize=(5,5))

ax.fill_between(q_hist.time,
                q_hist.sel(quantile=.05),
                q_hist.sel(quantile=.95),
                color="#616161", alpha=0.25)
ax.fill_between(q_ssp126.time,
                q_ssp126.sel(quantile=.05),
                q_ssp126.sel(quantile=.95),
                color="#0e57cc", alpha=0.25)
ax.fill_between(q_ssp370.time,
                q_ssp370.sel(quantile=.05),
                q_ssp370.sel(quantile=.95),
                color="#e81515", alpha=0.25)

ytop = 6
ax.set_ylabel("Relative to 1995-2014 (°C)")
ax.axhline(y=0, color="black")
ax.text(np.datetime64('2024-12-31T00:00:00.000000000'), -1, "Near\nterm")
ax.text(np.datetime64('2044-12-31T00:00:00.000000000'), -1, "Mid\nterm")
ax.text(np.datetime64('2084-12-31T00:00:00.000000000'), -1, "Long\nterm")
ax.fill_between(
    m_ssp126["time"].sel(time=slice("20200101", "20400101")),
    -2, ytop+.82, alpha=.33, color="grey")
ax.fill_between(
     m_ssp126["time"].sel(time=slice("20400101", "20600101")),
    -2, ytop+.82, alpha=.33, color="grey")
ax.fill_between(
     m_ssp126["time"].sel(time=slice("20800101", None)),
    -2, ytop+.82, alpha=.33, color="grey")

ax2 = ax.twinx()
ax2.set_ylabel("Relative to 1850-1900 (°C)")
ax2.set_ylim(bottom=-2+.82, top=ytop+.82)
ax.set_ylim([-2, ytop])

ax.plot(m_hist.time, m_hist, color="#010101", label=f"Historical  {len(un_mean_hist['member'])}")
ax.plot(m_ssp126.time, m_ssp126, color="#0e57cc", label=f"SSP1-2.6  {len(un_mean_ssp126['member'])}")
ax.plot(m_ssp245.time, m_ssp245, color="#e68d26", label=f"SSP2-4.5  {len(un_mean_ssp245['member'])}")
ax.plot(m_ssp370.time, m_ssp370, color="#e81515", label=f"SSP3-7.0  {len(un_mean_ssp370['member'])}")
ax.plot(m_ssp585.time, m_ssp585, color="#9b1a22", label=f"SSP5-8.5  {len(un_mean_ssp585['member'])}")

# ax2.plot(m_hist_18.time, m_hist_18, color="#010101", label=f"Historical  {len(mean_hist['member'])}")
# ax2.plot(m_ssp126_18.time, m_ssp126_18, color="#0e57cc", label=f"SSP1-2.6  {len(mean_ssp126['member'])}")
# ax2.plot(m_ssp245_18.time, m_ssp245_18, color="#e68d26", label=f"SSP2-4.5  {len(mean_ssp245['member'])}")
# ax2.plot(m_ssp370_18.time, m_ssp370_18, color="#e81515", label=f"SSP3-7.0  {len(mean_ssp370['member'])}")
# ax2.plot(m_ssp585_18.time, m_ssp585_18, color="#9b1a22", label=f"SSP5-8.5  {len(mean_ssp585['member'])}")

ax.margins(0)
ax2.margins(0)
ax.legend()

plt.savefig("pangeo-unweighted.pdf")

## Task graphs

In [ ]:
u = historical_dss[0]["tas"].isel(time=slice(0,221*3)).mean(["lat", "lon"])
w = historical_dss[0]["tas"].isel(time=slice(0,221*3)).weighted(
    np.cos(np.deg2rad(historical_dss[0].coords["lat"]))).mean(["lat", "lon"])

In [ ]:
u.data.visualize(filename="tg-unweighted.pdf", optimize_graph=True)

In [ ]:
w.data.visualize(filename="tg-weighted.pdf", optimize_graph=True)

To bebug using the distributed scheduler:

```python
from dask.distributed import Client, LocalCluster
from dask.distributed import performance_report

cluster = LocalCluster(processes=True,threads_per_worker=1)
client = Client(cluster)
```

# Measure performance

In [ ]:
from gsat import measure

In [ ]:
runs = 5
workers = [2,4,8]

In [ ]:
results = measure("weighted-pangeo", workers, runs, weighted,
                  historical_dss, ssp126_dss, ssp245_dss, ssp370_dss, ssp585_dss)

In [ ]:
pd.DataFrame.from_records(results).to_csv("weighted-pangeo.csv", index=False)